# T1564.006 - Run Virtual Instance
Adversaries may carry out malicious operations using a virtual instance to avoid detection. A wide variety of virtualization technologies exist that allow for the emulation of a computer or computing environment. By running malicious code inside of a virtual instance, adversaries can hide artifacts associated with their behavior from security tools that are unable to monitor activity inside the virtual instance. Additionally, depending on the virtual networking implementation (ex: bridged adapter), network traffic generated by the virtual instance can be difficult to trace back to the compromised host as the IP address and hostname might not match known values.(Citation: SingHealth Breach Jan 2019)

Adversaries may utilize native support for virtualization (ex: Hyper-V) or drop the necessary files to run a virtual instance (ex: VirtualBox binaries). After running a virtual instance, adversaries may create a shared folder between the guest and host with permissions that enable the virtual instance to interact with the host file system.(Citation: Sophos Ragnar May 2020)

## Atomic Tests

### Atomic Test #1 - Register Portable Virtualbox
ransomware payloads via virtual machines (VM). 
[Maze ransomware](https://threatpost.com/maze-ransomware-ragnar-locker-virtual-machine/159350/)

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: MSI file must exist on disk at specified location (#{msi_file_path})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1564.006\bin\Virtualbox_52.msi") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1564.006\bin\Virtualbox_52.msi") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1564.006/bin/Virtualbox_52.msi" -OutFile "PathToAtomicsFolder\T1564.006\bin\Virtualbox_52.msi"

```
##### Description: CAB file must exist on disk at specified location (#{cab_file_path})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1564.006\bin\common.cab") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1564.006\bin\common.cab") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1564.006/bin/common.cab" -OutFile "PathToAtomicsFolder\T1564.006\bin\common.cab" 

```
##### Description: Old version of Virtualbox must be installed

##### Check Prereq Commands:
```cmd
if (Test-Path "C:\Program Files\Oracle\VirtualBox\VboxC.dll") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
msiexec /i "PathToAtomicsFolder\T1564.006\bin\Virtualbox_52.msi" /qn

```

In [ ]:
Invoke-AtomicTest T1564.006 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"C:\Program Files\Oracle\VirtualBox\VBoxSVC.exe" /reregserver
regsvr32 /S "C:\Program Files\Oracle\VirtualBox\VboxC.dll"
rundll32 "C:\Program Files\Oracle\VirtualBox\VBoxRT.dll,RTR3Init"
sc create VBoxDRV binpath= "C:\Program Files\Oracle\VirtualBox\drivers\VboxDrv.sys" type= kernel start= auto error= normal displayname= PortableVBoxDRV
sc start VBoxDRV
```

In [ ]:
Invoke-AtomicTest T1564.006 -TestNumbers 1

#### Cleanup: 
```cmd
sc stop VBoxDRV
sc delete VBoxDRV
regsvr32 /u /S "C:\Program Files\Oracle\VirtualBox\VboxC.dll"
msiexec /x "PathToAtomicsFolder\T1564.006\bin\Virtualbox_52.msi" /qn
```

In [ ]:
Invoke-AtomicTest T1564.006 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Create and start VirtualBox virtual machine
Create a simple VirtualBox VM and start up the machine
Cleanup command stops and deletes the newly created VM and associated files
https://www.virtualbox.org/manual/ch08.html#vboxmanage-startvm
https://news.sophos.com/en-us/2020/05/21/ragnar-locker-ransomware-deploys-virtual-machine-to-dodge-security/
https://attack.mitre.org/techniques/T1564/006/

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: VirtualBox must exist on disk at specified locations (#{virtualbox_exe})

##### Check Prereq Commands:
```cmd
if (Test-Path "C:\Program Files\Oracle\VirtualBox\VirtualBox.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
$wc = New-Object System.Net.WebClient
$wc.DownloadFile("https://download.virtualbox.org/virtualbox/6.1.32/VirtualBox-6.1.32-149290-Win.exe","PathToAtomicsFolder\..\ExternalPayloads\VirtualBox-6.1.32-149290-Win.exe")
start-process -FilePath "PathToAtomicsFolder\..\ExternalPayloads\VirtualBox-6.1.32-149290-Win.exe" -ArgumentList "--silent" -Wait

```
##### Description: VBoxManage must exist on disk at specified locations (#{vboxmanage_exe})

##### Check Prereq Commands:
```cmd
if (Test-Path "C:\Program Files\Oracle\VirtualBox\VBoxManage.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
$wc = New-Object System.Net.WebClient
$wc.DownloadFile("https://download.virtualbox.org/virtualbox/6.1.32/VirtualBox-6.1.32-149290-Win.exe","PathToAtomicsFolder\..\ExternalPayloads\VirtualBox-6.1.32-149290-Win.exe")
start-process -FilePath "PathToAtomicsFolder\..\ExternalPayloads\VirtualBox-6.1.32-149290-Win.exe" -ArgumentList "--silent" -Wait

```

In [ ]:
Invoke-AtomicTest T1564.006 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"C:\Program Files\Oracle\VirtualBox\VBoxManage.exe" createvm --name "Atomic VM" --register
"C:\Program Files\Oracle\VirtualBox\VBoxManage.exe" modifyvm "Atomic VM" --firmware efi
"C:\Program Files\Oracle\VirtualBox\VBoxManage.exe" startvm "Atomic VM"
```

In [ ]:
Invoke-AtomicTest T1564.006 -TestNumbers 2

#### Cleanup: 
```cmd
"C:\Program Files\Oracle\VirtualBox\VBoxManage.exe" controlvm "Atomic VM" poweroff
"C:\Program Files\Oracle\VirtualBox\VBoxManage.exe" unregistervm "Atomic VM" --delete```

In [ ]:
Invoke-AtomicTest T1564.006 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Create and start Hyper-V virtual machine
Create a simple Hyper-V VM (Windows native hypervisor) and start up the machine
Cleanup command stops and deletes the newly created VM
https://docs.microsoft.com/en-us/virtualization/hyper-v-on-windows/quick-start/enable-hyper-v
https://embracethered.com/blog/posts/2020/shadowbunny-virtual-machine-red-teaming-technique/
https://attack.mitre.org/techniques/T1564/006/

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Hyper-V must be enabled on the system
Checks whether Hyper-V is enabled. If not, enables Hyper-V and forces a required restart

##### Check Prereq Commands:
```powershell
if ((Get-WindowsOptionalFeature -Online -FeatureName Microsoft-Hyper-V).State = "Enabled") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Enable-WindowsOptionalFeature -Online -FeatureName Microsoft-Hyper-V -All -Force

```

In [ ]:
Invoke-AtomicTest T1564.006 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$VM = "Atomic VM"
New-VM -Name $VM -Generation 2
Set-VMFirmware $VM -EnableSecureBoot Off
Start-VM $VM```

In [ ]:
Invoke-AtomicTest T1564.006 -TestNumbers 3

#### Cleanup: 
```powershell
Stop-VM $VM -Force
Remove-VM $VM -Force```

In [ ]:
Invoke-AtomicTest T1564.006 -TestNumbers 3 -Cleanup

## Detection
Consider monitoring for files and processes associated with running a virtual instance, such as binary files associated with common virtualization technologies (ex: VirtualBox, VMware, QEMU, Hyper-V). Consider monitoring the size of virtual machines running on the system. Adversaries may create virtual images which are smaller than those of typical virtual machines.(Citation: Shadowbunny VM Defense Evasion) Network adapter information may also be helpful in detecting the use of virtual instances.

Consider monitoring for process command-line arguments that may be atypical for benign use of virtualization software. Usage of virtualization binaries or command-line arguments associated with running a silent installation may be especially suspect (ex. <code>-silent</code>, <code>-ignore-reboot</code>), as well as those associated with running a headless (in the background with no UI) virtual instance (ex. <code>VBoxManage startvm $VM --type headless</code>).(Citation: Shadowbunny VM Defense Evasion) Similarly, monitoring command line arguments which suppress notifications may highlight potentially malicious activity (ex. <code>VBoxManage.exe setextradata global GUI/SuppressMessages "all"</code>).

Monitor for commands which enable hypervisors such as Hyper-V.  If virtualization software is installed by the adversary, the Registry may provide detection opportunities. Consider monitoring for [Windows Service](https://attack.mitre.org/techniques/T1543/003), with respect to virtualization software. 

Benign usage of virtualization technology is common in enterprise environments, data and events should not be viewed in isolation, but as part of a chain of behavior.